# AggChart
The AggChart is a special chart that displays aggregated data from another chart (a "control" chart).  The AggChart is hooked up with the control chart via sonar sender and receiver.  The AggChart can display data from the control chart aggregated on one or more operations, such as sum, avg, and count.

The AggChart acts like a regular JoogleChart in many ways, but is different in others.  An AggChart is created without data -- it gets its data from the control.  A filter cannot be added to the AggChart itself, and the AggChart can't take role or tooltip columns.  However, an AggChart can be of any chart type, and can be styled and formatted for layout and display.  (Note:  many Google Charts won't be compatible with the data model used in the AggChart.  The standard charts -- Table, LineChart, ColumnChart, etc. -- will work fine.  Other charts may throw an error.)

#### Filtering a control chart
A particular value of the AggChart is that it will automatically update if the control chart is filtered.  However, a filter is not required.  But if a filter is not added to the control chart, there's really nothing cool about the AggChart.

The control chart can be filtered either by adding the filter with **.add_filter()** or be hooking it up with a sonar sender and receiver.

# Usage: the control chart
The control chart is a normal JoogleChart.  It must be given the following sender.
#### .add_sender(key, on="ready", message_type="agg")
The key is an arbitrary string.  The same key must be used in the AggChart's receiver.

# Usage:  the AggChart
#### chart_type
Pass the chart_type to the constructor, as with a regular JoogleChart.

#### chart data:  DO NOT PASS DATA TO THE AggChart CONSTRUCTOR

### .add_receiver(key, group_column, agg_columns)
#### key
The key must be the same one used in the control chart.
#### group_column
An integer.  The group_column is the index of the column in the control chart on which data is grouped.  This column will be a category in the dataframe, and not a column of data.  For example, if you wanted the AggChart to show the average sales by country, the group_column would be the column with the countries.
#### agg_columns
agg_columns is a list of dictionaries.  Each dictionary specifies an aggregation to be displayed in the AggChart.  Each dictionary would have the following keys.
- ** column (required)**.  An integer.  This is the column of data in the control chart that is to be aggregated.  If the AggChart is to show average sales by country, the value is the index of the column with the sales data.
- ** function (required)**.  A string.  This is the name of the aggregation function, e.g., "sum", "count".  See a list of available functions below.
- ** label (optional) **.  A string.  The label is the display name of the series of aggregated data.  If a label is not provided, the name of the series will be simply be the name of the aggregating function.

# Available aggregation functions
A list of available functions is below.  Additional, custom functions can be added.  Consult your neighborhood Joogle developer.
- avg
- count
- sum
- max
- min


In [2]:
from jooglechart import JoogleChart, AggChart, Filter, ChartRow
from jooglechart.widgets import Box
from jooglechart.utils import get_df

# AggChart with native GoogleChart filtering

In [5]:
f = Filter("CategoryFilter")
f.add_options(filterColumnIndex=1)

df = get_df(rows=15, cols=1, category_column=True, category_column2=True)
chart = JoogleChart(df, chart_type="Table")
chart.add_filter(f)

chart.add_sender(key="AGG1", on="ready", message_type="agg")

agg = AggChart(chart_type="BarChart")
agg.add_chart_options(legend_position="top")

agg_columns = []
agg_columns.append({'column': 3, 'function': 'max'})
agg_columns.append({'column': 3, 'function': 'min'})
agg_columns.append({'column': 3, 'function': 'avg'})

agg.add_receiver(key="AGG1", group_column=2, agg_columns=agg_columns)

row = ChartRow(chart, agg)

row.show()

# AggChart with sonar filter on control chart

In [4]:
f = Filter("CategoryFilter", data=df)
f.add_options(filterColumnIndex=1)
f.add_sender(key="FILTER")

df = get_df(rows=15, cols=2, category_column=True, category_column2=True)
chart = JoogleChart(df, chart_type="Table")
chart.add_receiver(key="FILTER", column=1)

chart.add_sender(key="AGG2", on="ready", message_type="agg")

agg = AggChart(chart_type="Table")

agg_columns = []
agg_columns.append({'column': 3, 'function': 'count', 'label': "A Count"})
agg_columns.append({'column': 3, 'function': 'sum', 'label': "A Total"})
agg_columns.append({'column': 3, 'function': 'avg', 'label': "A Average"})
agg_columns.append({'column': 4, 'function': 'count', 'label': "B Count"})
agg_columns.append({'column': 4, 'function': 'sum', 'label': "B Total"})
agg_columns.append({'column': 4, 'function': 'avg', 'label': "B Average"})

agg.add_receiver(key="AGG2", group_column=2, agg_columns=agg_columns)

row = ChartRow(chart, agg)

box = Box(f, row)

box.show()